In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
poli = pd.read_csv('C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/countypres_2000-2020.csv', converters={'county_fips': str})

In [3]:
poli.columns

Index(['year', 'state', 'state_po', 'county_name', 'county_fips', 'office',
       'candidate', 'party', 'candidatevotes', 'totalvotes', 'version',
       'mode'],
      dtype='object')

In [4]:
poli.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,1001,US PRESIDENT,RALPH NADER,GREEN,160,17208,20220315,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,1001,US PRESIDENT,OTHER,OTHER,113,17208,20220315,TOTAL
4,2000,ALABAMA,AL,BALDWIN,1003,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL


In [5]:
poli = poli.drop(columns={'state', 'state_po', 'county_name', 'office',
       'candidate', 'totalvotes', 'version',
       'mode'})

In [6]:
poli['county_fips'] = poli['county_fips'].apply(lambda x: str(x).zfill(5))

In [7]:
poli = poli.rename(columns={'county_fips': 'FIPS'})

In [8]:
# Step 1: Calculate the maximum candidatevotes for each group of 'year' and 'FIPS'
max_votes = poli.groupby(['year', 'FIPS'])['candidatevotes'].max().reset_index()

# Step 2: Merge this information back to the original DataFrame to filter rows
# This operation joins the original DataFrame with the max_votes DataFrame on the specified keys and 'candidatevotes',
# effectively filtering the original DataFrame to only include rows with maximum votes.
poli_max_votes = poli.merge(max_votes, on=['year', 'FIPS', 'candidatevotes'])

In [9]:
poli_max_votes.shape

(18975, 4)

In [10]:
poli_max_votes = poli_max_votes[(poli_max_votes['year'] == 2012) | (poli_max_votes['year'] == 2016) | (poli_max_votes['year'] == 2020)]

In [11]:
# Dropping duplicate entries, keeping the first occurrence
poli_max_votes_unique = poli_max_votes.drop_duplicates(subset=['year', 'FIPS'])

# Now, try to pivot again
df_pivoted = poli_max_votes_unique.pivot(index='FIPS', columns='year', values='party')
df_pivoted.columns = [f'PresW_{col}' for col in df_pivoted.columns]
df_pivoted.reset_index(inplace=True)

In [12]:
df_pivoted.head(10)

,FIPS,PresW_2012,PresW_2016,PresW_2020
0,000NA,DEMOCRAT,DEMOCRAT,DEMOCRAT
1,01001,REPUBLICAN,REPUBLICAN,REPUBLICAN
2,01003,REPUBLICAN,REPUBLICAN,REPUBLICAN
3,01005,DEMOCRAT,REPUBLICAN,REPUBLICAN
4,01007,REPUBLICAN,REPUBLICAN,REPUBLICAN
5,01009,REPUBLICAN,REPUBLICAN,REPUBLICAN
6,01011,DEMOCRAT,DEMOCRAT,DEMOCRAT
7,01013,REPUBLICAN,REPUBLICAN,REPUBLICAN
8,01015,REPUBLICAN,REPUBLICAN,REPUBLICAN
9,01017,REPUBLICAN,REPUBLICAN,REPUBLICAN


In [13]:
mask = df_pivoted['FIPS'].str.contains('NA', na=False)

# Inverting the mask to keep rows where 'FIPS' does NOT contain 'NA'
df_pivoted = df_pivoted[~mask]

In [14]:
df_poli = df_pivoted
df_poli.shape
df_poli.head(10)
df_poli = df_poli.reset_index(drop=True)

In [15]:
df_poli.head()

,FIPS,PresW_2012,PresW_2016,PresW_2020
0,01001,REPUBLICAN,REPUBLICAN,REPUBLICAN
1,01003,REPUBLICAN,REPUBLICAN,REPUBLICAN
2,01005,DEMOCRAT,REPUBLICAN,REPUBLICAN
3,01007,REPUBLICAN,REPUBLICAN,REPUBLICAN
4,01009,REPUBLICAN,REPUBLICAN,REPUBLICAN


In [17]:
df_poli.to_csv('poli_reformed2.csv')